In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm, tqdm_notebook

from wavefunction1d import solve_eigenproblem

pi = np.pi

In [3]:
# QuTip
from qutip import Options, Qobj, basis, destroy, create, qeye, tensor, Cubic_Spline

In [2]:
phi_max, Nphi = 10*np.pi, 501
phi = np.linspace(-phi_max, phi_max, Nphi)

dphi = 2*phi_max/Nphi 

K_mtx = np.diag( -2* np.ones(Nphi) ) + np.diag( np.ones(Nphi - 1 ) , 1) + np.diag( np.ones(Nphi - 1), -1)

Ec = 30
EL = 0.5
Ej = 10

phiext_list = np.linspace(0 , np.pi/2, 21)

psi0s = []
psi1s = []


V = Ej* np.cos(phi - 1*np.pi/4)  + EL*(phi)**2   

H = -Ec*K_mtx/dphi**2 + np.diag(V)

evals, evecs = solve_eigenproblem(H)

fig, ax = plt.subplots()


a = -50

psi0 = a*evecs[0] +  evals[0]
psi1 = a*evecs[1] +  evals[1]



ax.plot(phi, psi0)
ax.plot(phi, -psi1)

ax2 = ax.twinx()
# ax2.plot(phi[400:-400], V[400:-400], '--b', alpha = 0.2)
ax.plot(phi[abs(phi)<7], V[abs(phi)<7], '--b', alpha = 0.2)


FigureCanvasNbAgg()

## lambds scan

In [3]:
phi_max, Nphi = 10*np.pi, 501
phi = np.linspace(-phi_max, phi_max, Nphi)

dphi = 2*phi_max/Nphi 

K_mtx = np.diag( -2* np.ones(Nphi) ) + np.diag( np.ones(Nphi - 1 ) , 1) + np.diag( np.ones(Nphi - 1), -1)

Ec = 15
EL = 0.5
Ej = 15


Nframes = 201
phiext_list = np.linspace(-np.pi/2, np.pi/4, Nframes)

psi0s = []
psi1s = []

E0s = []
E1s = []

Vs = []

a = -50

tphiext_list = tqdm_notebook(phiext_list)

for phiext in tphiext_list:
    V = Ej* np.cos(phi - phiext)  + EL*(phi)**2   

    H = -Ec*K_mtx/dphi**2 + np.diag(V)

    evals, evecs = solve_eigenproblem(H)

    psi0 = -a*evecs[0] 
    psi1 = a*evecs[1] 

    E0 =  evals[0]
    E1 =  evals[1]

    
    psi0s.append(psi0)
    psi1s.append(psi1)

    E0s.append( E0 )
    E1s.append( E1 )
    
    Vs.append( V )
    

In [11]:
from matplotlib.animation import FuncAnimation, PillowWriter

fig, (ax, ax2) = plt.subplots(1,2, figsize=(7, 4), sharey = False)


ax.set_xticklabels([])
ax.set_yticklabels([])

ax2.set_xticklabels([])
ax2.set_yticklabels([])




line1, = ax.plot(phi /2/np.pi,  psi0s[0] + E0s[0], c = 'C0')
line2, = ax.plot(phi /2/np.pi,  psi1s[0] + E1s[0], c = 'C1')




ax2.plot(phiext_list, E0s )
ax2.plot(phiext_list, E1s )

m1, = ax2.plot([phiext_list[0]],  [E0s[0]], c = 'C0', marker = 'x')
m2, = ax2.plot([phiext_list[1]],  [E1s[1]], c = 'C1', marker = 'x')

line3, = ax.plot(phi /2/np.pi, Vs[0], '--k', alpha = 0.2)

signs = [np.sign(psi0s[0][Nphi//2+15]),
         np.sign(psi0s[1][Nphi//2+15])]

ax.set_xlim(-2.25,2.25)
ax.set_ylim(-20,25)

plt.tight_layout()

lines = [line1, line2, line3]

markers = [m1, m2]

psis = [psi0s, psi1s, Vs]

Es = [E0s, E1s]

def update(j):
    
    for l, ps,E, sign in zip(lines, psis,Es, signs):
        psi = np.array(ps[j])
        x = phi /2/pi

        y = 1*psi*sign*np.sign(psi[Nphi//2+15])
    
        l.set_data(x , y + E[j])
        
    lines[2].set_data(x , Vs[j])
    
    
    for m, E in zip(markers, Es):
         
        x = phiext_list[j]

        y = E[j]
    
        m.set_data(x , y)
     

    return lines, markers, ax, ax2


    
    
anim = FuncAnimation(fig, update, frames=np.arange(0, 2*Nframes//3), interval=20,repeat=False)

anim.save('wf.gif')

FigureCanvasNbAgg()

MovieWriter ffmpeg unavailable; trying to use <class 'matplotlib.animation.PillowWriter'> instead.


In [78]:
plt.close('all')

## EL scan

In [6]:
201//4

50

In [12]:
phi_max, Nphi = 10*np.pi, 201
phi = np.linspace(-phi_max, phi_max, Nphi)

dphi = 2*phi_max/Nphi 

K_mtx = np.diag( -2* np.ones(Nphi) ) + np.diag( np.ones(Nphi - 1 ) , 1) + np.diag( np.ones(Nphi - 1), -1)

Ec = 15
# EL = 0.5
phiext0 = -np.pi/4


Ej = 15


Nframes = 201


EL_list = np.logspace(-0.3, -3, Nframes)
phiext_list = np.linspace(-np.pi/2, np.pi/4, 101)

psi0s = []
psi1s = []

E0s = []
E1s = []



band0s = []
band1s = []

Vs = []

a = -50

tEL_list = tqdm_notebook(EL_list)

for EL in tEL_list:
    V = Ej* np.cos(phi - phiext0)  + EL*(phi)**2   

    H = -Ec*K_mtx/dphi**2 + np.diag(V)

    evals, evecs = solve_eigenproblem(H)

    psi0 = -a*evecs[0] 
    psi1 = a*evecs[1] 

    E0 =  evals[0]
    E1 =  evals[1]

    
    psi0s.append(psi0)
    psi1s.append(psi1)

    E0s.append( E0 )
    E1s.append( E1 )
    
    Vs.append( V )
    
    
    band0 = []
    band1 = []
    for phiext in phiext_list:
        V = Ej* np.cos(phi - phiext)  + EL*(phi)**2   

        H = -Ec*K_mtx/dphi**2 + np.diag(V)

        evals, evecs = solve_eigenproblem(H)
        
        band0.append(evals[0])
        band1.append(evals[1])
    
    band0s.append(band0)
    band1s.append(band1)


In [14]:
from matplotlib.animation import FuncAnimation, PillowWriter

fig, (ax, ax2) = plt.subplots(1,2, figsize=(7, 4), sharey = False)


ax.set_xticklabels([])
ax.set_yticklabels([])

ax2.set_xticklabels([])
ax2.set_yticklabels([])

line1, = ax.plot(phi /2/np.pi,  psi0s[0] + 0*E0s[0], c = 'C0')
line2, = ax.plot(phi /2/np.pi,  psi1s[0] + 0*E1s[0], c = 'C1')




line21, = ax2.plot(phiext_list, band0s[0] - band0s[0][0]  )
line22, = ax2.plot(phiext_list, band1s[1] - band0s[0][0] )


m1, = ax2.plot([phiext0],  [E0s[0] - band0s[0][0]], 'x', c = 'C0')
m2, = ax2.plot([phiext0],  [E1s[0] - band0s[0][0]], 'x', c = 'C1')

# m1, = ax2.plot([phiext_list[0]],  [E0s[0]], c = 'C0', marker = 'x')
# m2, = ax2.plot([phiext_list[1]],  [E1s[1]], c = 'C1', marker = 'x')

line3, = ax.plot(phi /2/np.pi, Vs[0] - E0s[0], '--k', alpha = 0.2)

signs = [np.sign(psi0s[0][Nphi//2+15]),
         -np.sign(psi0s[1][Nphi//2+15])]

ax.set_xlim(-2.25,2.25)
ax.set_ylim(-20,25)

plt.tight_layout()

lines = [line1, line2, line3]

line2s = [line21, line22]

markers = [m1, m2]

psis = [psi0s, psi1s, Vs]

Es = [E0s, E1s]

bands = [band0s, band1s]

def update(j):
    
    for l, ps,E, sign in zip(lines, psis,Es, signs):
        psi = np.array(ps[j])
        x = phi /2/pi

        y = 1*psi*sign*np.sign(psi[Nphi//2+15])
    
        l.set_data(x , y + 0*E[j])
        
    lines[2].set_data(x , Vs[j] - E[j])
    
    
    for l, band in zip(line2s, bands):
         
        x = phiext_list

        y = np.array(band[j]) - np.array(band0s)[j,0]
    
        l.set_data(x , y)

        
        
    for m, E in zip(markers, Es):
         
        x = [phiext0]

        y = [E[j] - band0s[j][0]]
    
        m.set_data(x , y)        
        

    return lines, markers, ax, ax2


    
    
anim = FuncAnimation(fig, update, frames=np.arange(0, Nframes), interval=20, repeat=False)

anim.save('wf1.gif')

FigureCanvasNbAgg()

MovieWriter ffmpeg unavailable; trying to use <class 'matplotlib.animation.PillowWriter'> instead.
